In [2]:
# import the neccessary libs
import os
import numpy as np 
import librosa
import hmmlearn.hmm as hmm

`Step 1: Collect a dataset of audio recordings of each command to recognize`

In [11]:
# trial 1: Error: We end up with a directory instead of a file when executing the for commans in commands
#                 loop: therefore, we used an alternative folder with few datasamples in a single dir


# dataset_path = "/home/ix502iv/Documents/Audio_Trad/HMM/custom_commands" # use absolute path
# commands = ["demo_fider_ac", "demo_fider_bilgileri", "demo_fider_kapat", "nem_durumu", "gsm_durumu"]
# dataset = {}

In [3]:
# trial 2 : made use of a new dataset with very little data smaples

dataset_path = "/home/ix502iv/Documents/Audio_Trad/HMM/custom_commands_copy" # use absolute path
commands = ["demo_fider_ac", "demo_fider_bilgileri", "demo_fider_kapat", "nem_durumu", "gsm_durumu"]
dataset = {}

In [5]:
for command in commands:
    command_path = os.path.join(dataset_path, command)
    # print(command_path)
    files = os.listdir(command_path)
    print(files)
    recordings = []
    for file in files:
        file_path = os.path.join(command_path, file)
        # print(file_path)
        # we've moved one file: demo fider ac into a directory above: remeber to delete it
        recording, sr = librosa.load(file_path, sr=None)
        # print(recording, sr)
        recordings.append(recording)
    dataset[command] = recordings

['demo_fider_ac.wav']
['demo_fider_bilgileri.wav']
['demo_fider_kapat.wav']
['nem_durumu.wav']
['gsm_durumu.wav']


In [14]:
# Extract MFCC Features
def extract_features(recording):
    mfcc = librosa.feature.mfcc(y=recording, sr=sr, n_mfcc=13)
    print(mfcc)
    return mfcc.T

In [15]:
# Define States and Build the Model
n_states = 16 # the number of commands we have
# model = hmm.GaussianHMM(n_components=n_states)
model = hmm.GaussianHMM(n_components=4, covariance_type="diag", n_iter=1000)


In [16]:
%%capture
# Train the model on our data
for command in commands:
    X = np.vstack(
        [extract_features(recording) for recording in dataset[command]]
    ) # stacks arrays in sequence vertically: row-wise
    model.fit(X)

In [17]:
# test the model
test_file_path = "/home/ix502iv/Documents/Audio_Trad/HMM/custom_commands_copy/demo_fider_bilgileri/demo_fider_bilgileri.wav"
test_recording, sr = librosa.load(test_file_path, sr=None)
test_features = extract_features(test_recording)

[[-615.23425   -595.3319    -593.021     ... -599.8126    -598.0717
  -599.38586  ]
 [  70.48137     84.05586     82.83789   ...   80.06057     81.20296
    79.717804 ]
 [  34.989197    28.601088    23.122593  ...   27.707083    26.511002
    25.149128 ]
 ...
 [  -3.3713937   -2.7822943   -3.1301765 ...   -8.360065    -5.44553
    -5.4415436]
 [  -3.2295861   -3.7801251   -5.2786493 ...  -10.048229    -6.7899256
    -5.8770657]
 [  -1.5549121   -2.9548712   -3.2821972 ...   -4.0869446   -3.0633922
    -5.0855045]]


In [18]:
scores = {}
for command in commands:
    score = model.score(test_features) # compute the log probability under the model
    scores[command] = score

predicted_command = max(scores, key=scores.get)
print("Predicted Command: ", predicted_command)

Predicted Command:  demo_fider_ac
